## Neural Network

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.nn.Conv2d(channels, output, filter_height, filter_width)

torch.nn.Linear(in_features, out_features, bias = True)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input channel, 6 output channels, 5*5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### PyTorch下神经网络结构可视化方法：TensorboardX
需要Tf和tensorboardX

In [4]:
from tensorboardX import SummaryWriter

dummy_input = torch.rand(1,1,32,32)
model = Net()

with SummaryWriter(comment='Net') as w:
    w.add_graph(model,(dummy_input,))# 在同目录下生成runs文件夹
    # 在runs同级目录下使用命令行：tensorboard --logdir runs
    # 用浏览器打开生成的xxxx6006地址即可

一个模型可训练的参数可以通过调用 net.parameters() 返回：

In [5]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's.weight

10
torch.Size([6, 1, 5, 5])


试随机生成一个32*32的输入,把所有参数梯度缓存器置零,用随机的梯度来反向传播

In [6]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

net.zero_grad()
out.backward(torch.randn(1, 10))

tensor([[-0.0002,  0.1735,  0.0166,  0.0789, -0.1748, -0.0786,  0.0561, -0.0283,
          0.0327, -0.1336]], grad_fn=<AddmmBackward>)


### Loss function

In [7]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

tensor(1.8611, grad_fn=<MseLossBackward>)


跟随loss到反向传播路径，可以使用.grad_fn属性，得到一个计算图

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d

 -> view -> linear -> relu -> linear -> relu -> linear
 
 -> MSELoss
 
 -> loss

In [12]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

### 反向传播
需要清空现存的梯度，要不然梯度将会和现存的梯度累计到一起

In [13]:
net.zero_grad() # zeroes the gradient buffers of all parameters
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0062, -0.0005,  0.0087,  0.0111,  0.0142, -0.0140])


### 更新网络参数

eg：随机梯度下降 $weight = weight - learning_{rate} \cdot gradient$

In [14]:
learning_rate = 0.01
for f in net.parameters():
     f.data.sub_(f.grad.data * learning_rate)

额外的更新规则在torch.optim包中，如SGD，Nesterov-SGD，Adam，RMSProp等

In [ ]:
import torch.optim as optim
# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)
# in your training loop:
optimizer.zero_grad() # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # Does the update